#  Scraper allrecipes
### Categoria, Nombre, URL general, Position, URL de cada position, Breadcrumb, Image URLs.

In [ ]:
import requests
from bs4 import BeautifulSoup
import json

def extract_category_urls(url):
    # Realizar la solicitud HTTP GET
    response = requests.get(url)

    # Parsear el contenido HTML de la respuesta
    soup = BeautifulSoup(response.content, "html.parser")

    # Encontrar todos los elementos <div> con la clase "alphabetical-list__group"
    category_groups = soup.find_all("div", class_="alphabetical-list__group")

    category_data = []
    for category_group in category_groups:
        # Obtener el nombre de la categoría
        category_name = category_group.find("h3").text.strip()

        # Obtener los elementos <a> de la categoría
        category_items = category_group.find_all("a")
        for item in category_items:
            item_name = item.text.strip()
            item_url = item["href"]
            category_data.append((category_name, item_name, item_url))

    return category_data


def extract_breadcrumb_and_image_urls(url):
    # Realizar la solicitud HTTP GET a la URL extraída
    response = requests.get(url)

    # Parsear el contenido HTML de la respuesta
    soup = BeautifulSoup(response.content, "html.parser")

    # Encontrar el elemento <script> que contiene el ld+json
    script_tag = soup.find("script", type="application/ld+json")

    # Obtener el contenido del elemento <script>
    script_content = script_tag.string

    # Convertir el contenido a un objeto JSON
    data = json.loads(script_content)

    # Acceder a la información deseada
    main_entity = data[0]["mainEntityOfPage"]
    breadcrumb_list = main_entity["breadcrumb"]["itemListElement"]

    breadcrumbs = []
    # Obtener el ListItem anterior al último ListItem
    if len(breadcrumb_list) >= 2:
        penultimate_item = breadcrumb_list[-2]
        breadcrumb = penultimate_item.get("item", {}).get("@id", "").split("https://www.allrecipes.com/recipes/")[-1].split("/")[1:-1]
        if breadcrumb:
            breadcrumbs = breadcrumb

    # Encontrar todas las etiquetas <img> en la página
    img_tags = soup.find_all("img")

    # Obtener las URLs de las imágenes
    image_urls = [img.get("src") for img in img_tags if img.get("src")]
    if image_urls:
        image_urls = [image_urls[0]]

    return breadcrumbs, image_urls

# URL de la página inicial para extraer las categorías
initial_url = "https://www.allrecipes.com/ingredients-a-z-6740416"

# Paso 1: Extraer todas las URL de las categorías y sus nombres
category_data = extract_category_urls(initial_url)

# Paso 2: Para cada categoría y URL, extraer las URL internas, el Breadcrumb y las URLs de las imágenes
all_breadcrumbs = []
for category_name, item_name, item_url in category_data:
    print("Categoría:", category_name)
    print("Nombre:", item_name)
    print("URL:", item_url)

    # Realizar la solicitud HTTP GET
    response = requests.get(item_url)

    # Parsear el contenido HTML de la respuesta
    soup = BeautifulSoup(response.content, "html.parser")

    # Encontrar el elemento <script> que contiene el ld+json
    script_tag = soup.find("script", type="application/ld+json")

    # Obtener el contenido del elemento <script>
    script_content = script_tag.string

    # Convertir el contenido a un objeto JSON
    data = json.loads(script_content)

    # Acceder a la información deseada
    item_list = data[0]["itemListElement"]

    for item in item_list:
        position = item["position"]
        url = item["url"]

        print("Position:", position)
        print("URL:", url)

        # Obtener el Breadcrumb y las URLs de las imágenes
        breadcrumbs, image_urls = extract_breadcrumb_and_image_urls(url)

        if breadcrumbs:
            print("Breadcrumb:", breadcrumbs)

        if image_urls:
            print("Image URLs:", image_urls)

        print()

        # Descargar las imágenes si se desea
        # for i, image_url in enumerate(image_urls):
        #     response = requests.get(image_url)
        #     with open(f"image_{position}_{i+1}.jpg", "wb") as f:
        #         f.write(response.content)

    print()

print("-----------------------Breadcrumbs:-----------------------")
for breadcrumb in all_breadcrumbs:
    print(breadcrumb)
